In [1]:
%load_ext autoreload

In [2]:
import itertools

import numpy as np
import pandas as pd
pd.set_option("max_columns", None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from scipy.spatial.distance import jensenshannon
from scipy import signal
from scipy.special import rel_entr

from time_delay_embedding import time_delay_embedding
from feature_extraction import covariance, co_integration, covariance

## Baseline

### data

In [3]:
buoy = pd.read_csv(
    "data/smart_buoy.csv",
    skiprows = [1],
    parse_dates = ["time"],
)
buoy.set_index("time", inplace = True)
buoy = buoy.resample("H").mean()
buoy.columns = [
    "PeakP",
    "PeakD",
    "Upcross",
    "SWH",
    "SeaTemp",
    "Hmax",
    "THmax",
    "MCurDir",
    "MCurSpd",
]
buoy.head()

,PeakP,PeakD,Upcross,SWH,SeaTemp,Hmax,THmax,MCurDir,MCurSpd
time,,,,,,,,,
2022-01-01 00:00:00+00:00,4.010,162.153850,3.890,146.0,10.130833,267.5,4.675,178.051281,0.137000
2022-01-01 01:00:00+00:00,4.130,161.362635,3.925,142.5,10.135000,206.0,4.080,179.882784,0.126000
2022-01-01 02:00:00+00:00,4.085,158.461540,3.855,143.0,10.118333,257.5,4.535,295.648351,0.178667
2022-01-01 03:00:00+00:00,5.985,191.428575,4.075,153.5,10.143333,260.0,8.170,339.413908,0.224333
2022-01-01 04:00:00+00:00,5.970,221.186815,4.305,171.0,10.143333,292.0,4.995,288.586082,0.211667


In [4]:
buoy.shape

(7835, 9)

### AR-Auto-Regressive Distributed Lags

In [5]:
# lagged features using time delay embedding
buoy_ds = []
for col in buoy:
    col_df = time_delay_embedding(buoy[col], n_lags = 24, horizon = 12)
    buoy_ds.append(col_df)
buoy_df = pd.concat(buoy_ds, axis = 1).dropna()
buoy_df.head()

,PeakP(t-23),PeakP(t-22),PeakP(t-21),PeakP(t-20),PeakP(t-19),PeakP(t-18),PeakP(t-17),PeakP(t-16),PeakP(t-15),PeakP(t-14),PeakP(t-13),PeakP(t-12),PeakP(t-11),PeakP(t-10),PeakP(t-9),PeakP(t-8),PeakP(t-7),PeakP(t-6),PeakP(t-5),PeakP(t-4),PeakP(t-3),PeakP(t-2),PeakP(t-1),PeakP(t-0),PeakP(t+1),PeakP(t+2),PeakP(t+3),PeakP(t+4),PeakP(t+5),PeakP(t+6),PeakP(t+7),PeakP(t+8),PeakP(t+9),PeakP(t+10),PeakP(t+11),PeakP(t+12),PeakD(t-23),PeakD(t-22),PeakD(t-21),PeakD(t-20),PeakD(t-19),PeakD(t-18),PeakD(t-17),PeakD(t-16),PeakD(t-15),PeakD(t-14),PeakD(t-13),PeakD(t-12),PeakD(t-11),PeakD(t-10),PeakD(t-9),PeakD(t-8),PeakD(t-7),PeakD(t-6),PeakD(t-5),PeakD(t-4),PeakD(t-3),PeakD(t-2),PeakD(t-1),PeakD(t-0),PeakD(t+1),PeakD(t+2),PeakD(t+3),PeakD(t+4),PeakD(t+5),PeakD(t+6),PeakD(t+7),PeakD(t+8),PeakD(t+9),PeakD(t+10),PeakD(t+11),PeakD(t+12),Upcross(t-23),Upcross(t-22),Upcross(t-21),Upcross(t-20),Upcross(t-19),Upcross(t-18),Upcross(t-17),Upcross(t-16),Upcross(t-15),Upcross(t-14),Upcross(t-13),Upcross(t-12),Upcross(t-11),Upcross(t-10),Upcross(t-9),Upcross(t-8),Upcross(t-7),Upcross(t-6),Upcross(t-5),Upcross(t-4),Upcross(t-3),Upcross(t-2),Upcross(t-1),Upcross(t-0),Upcross(t+1),Upcross(t+2),Upcross(t+3),Upcross(t+4),Upcross(t+5),Upcross(t+6),Upcross(t+7),Upcross(t+8),Upcross(t+9),Upcross(t+10),Upcross(t+11),Upcross(t+12),SWH(t-23),SWH(t-22),SWH(t-21),SWH(t-20),SWH(t-19),SWH(t-18),SWH(t-17),SWH(t-16),SWH(t-15),SWH(t-14),SWH(t-13),SWH(t-12),SWH(t-11),SWH(t-10),SWH(t-9),SWH(t-8),SWH(t-7),SWH(t-6),SWH(t-5),SWH(t-4),SWH(t-3),SWH(t-2),SWH(t-1),SWH(t-0),SWH(t+1),SWH(t+2),SWH(t+3),SWH(t+4),SWH(t+5),SWH(t+6),SWH(t+7),SWH(t+8),SWH(t+9),SWH(t+10),SWH(t+11),SWH(t+12),SeaTemp(t-23),SeaTemp(t-22),SeaTemp(t-21),SeaTemp(t-20),SeaTemp(t-19),SeaTemp(t-18),SeaTemp(t-17),SeaTemp(t-16),SeaTemp(t-15),SeaTemp(t-14),SeaTemp(t-13),SeaTemp(t-12),SeaTemp(t-11),SeaTemp(t-10),SeaTemp(t-9),SeaTemp(t-8),SeaTemp(t-7),SeaTemp(t-6),SeaTemp(t-5),SeaTemp(t-4),SeaTemp(t-3),SeaTemp(t-2),SeaTemp(t-1),SeaTemp(t-0),SeaTemp(t+1),SeaTemp(t+2),SeaTemp(t+3),SeaTemp(t+4),SeaTemp(t+5),SeaTemp(t+6),SeaTemp(t+7),SeaTemp(t+8),SeaTemp(t+9),SeaTemp(t+10),SeaTemp(t+11),SeaTemp(t+12),Hmax(t-23),Hmax(t-22),Hmax(t-21),Hmax(t-20),Hmax(t-19),Hmax(t-18),Hmax(t-17),Hmax(t-16),Hmax(t-15),Hmax(t-14),Hmax(t-13),Hmax(t-12),Hmax(t-11),Hmax(t-10),Hmax(t-9),Hmax(t-8),Hmax(t-7),Hmax(t-6),Hmax(t-5),Hmax(t-4),Hmax(t-3),Hmax(t-2),Hmax(t-1),Hmax(t-0),Hmax(t+1),Hmax(t+2),Hmax(t+3),Hmax(t+4),Hmax(t+5),Hmax(t+6),Hmax(t+7),Hmax(t+8),Hmax(t+9),Hmax(t+10),Hmax(t+11),Hmax(t+12),THmax(t-23),THmax(t-22),THmax(t-21),THmax(t-20),THmax(t-19),THmax(t-18),THmax(t-17),THmax(t-16),THmax(t-15),THmax(t-14),THmax(t-13),THmax(t-12),THmax(t-11),THmax(t-10),THmax(t-9),THmax(t-8),THmax(t-7),THmax(t-6),THmax(t-5),THmax(t-4),THmax(t-3),THmax(t-2),THmax(t-1),THmax(t-0),THmax(t+1),THmax(t+2),THmax(t+3),THmax(t+4),THmax(t+5),THmax(t+6),THmax(t+7),THmax(t+8),THmax(t+9),THmax(t+10),THmax(t+11),THmax(t+12),MCurDir(t-23),MCurDir(t-22),MCurDir(t-21),MCurDir(t-20),MCurDir(t-19),MCurDir(t-18),MCurDir(t-17),MCurDir(t-16),MCurDir(t-15),MCurDir(t-14),MCurDir(t-13),MCurDir(t-12),MCurDir(t-11),MCurDir(t-10),MCurDir(t-9),MCurDir(t-8),MCurDir(t-7),MCurDir(t-6),MCurDir(t-5),MCurDir(t-4),MCurDir(t-3),MCurDir(t-2),MCurDir(t-1),MCurDir(t-0),MCurDir(t+1),MCurDir(t+2),MCurDir(t+3),MCurDir(t+4),MCurDir(t+5),MCurDir(t+6),MCurDir(t+7),MCurDir(t+8),MCurDir(t+9),MCurDir(t+10),MCurDir(t+11),MCurDir(t+12),MCurSpd(t-23),MCurSpd(t-22),MCurSpd(t-21),MCurSpd(t-20),MCurSpd(t-19),MCurSpd(t-18),MCurSpd(t-17),MCurSpd(t-16),MCurSpd(t-15),MCurSpd(t-14),MCurSpd(t-13),MCurSpd(t-12),MCurSpd(t-11),MCurSpd(t-10),MCurSpd(t-9),MCurSpd(t-8),MCurSpd(t-7),MCurSpd(t-6),MCurSpd(t-5),MCurSpd(t-4),MCurSpd(t-3),MCurSpd(t-2),MCurSpd(t-1),MCurSpd(t-0),MCurSpd(t+1),MCurSpd(t+2),MCurSpd(t+3),MCurSpd(t+4),MCurSpd(t+5),MCurSpd(t+6),MCurSpd(t+7),MCurSpd(t+8),MCurSpd(t+9),MCurSpd(t+10),MCurSpd(t+11),MCurSpd(t+12)
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [6]:
buoy_df.shape

(4790, 324)

### 数据分割

In [7]:
# 预测变量
predictor_variables = buoy_df.columns.str.contains("\(t\-")
X = buoy_df.iloc[:, predictor_variables]
# print(X.head())
print(X.shape)

(4790, 216)


In [8]:
# 目标变量
target_var = "SWH"
target_variables = buoy_df.columns.str.contains(f"{target_var}\(t\+")
Y = buoy_df.iloc[:, target_variables]
# print(Y.head())
print(Y.shape)

(4790, 12)


In [9]:
# 训练集-测试集分割
X_train, X_test, Y_train, Y_test = train_test_split(
    X, 
    Y, 
    test_size = 0.3, 
    shuffle = False
)

### 模型构建

In [10]:
# 模型拟合
model_wo_fe = MultiOutputRegressor(LGBMRegressor())
model_wo_fe.fit(X_train, Y_train)

MultiOutputRegressor(estimator=LGBMRegressor())

In [11]:
# 模型预测
preds_wo_fe = model_wo_fe.predict(X_test)
print(preds_wo_fe.shape)

(1437, 12)


In [12]:
# 模型评估
mape(Y_test, preds_wo_fe)

0.23879314909803728

## Feature Engineering

### 单变量特征提取

In [13]:
SUMMARY_STATS = {
    "mean": np.mean,
    "sdev": np.std,
    "median": np.median,
}

univariate_features = {}
for col in buoy.columns:
    X_col = X.iloc[:, X.columns.str.startswith(col)]
    for feature, func in SUMMARY_STATS.items():
        univariate_features[f"{col}_{feature}"] = X_col.apply(func, axis = 1)
univariate_features_df = pd.concat(univariate_features, axis = 1)
univariate_features_df.head()

,PeakP_mean,PeakP_sdev,PeakP_median,PeakD_mean,PeakD_sdev,PeakD_median,Upcross_mean,Upcross_sdev,Upcross_median,SWH_mean,SWH_sdev,SWH_median,SeaTemp_mean,SeaTemp_sdev,SeaTemp_median,Hmax_mean,Hmax_sdev,Hmax_median,THmax_mean,THmax_sdev,THmax_median,MCurDir_mean,MCurDir_sdev,MCurDir_median,MCurSpd_mean,MCurSpd_sdev,MCurSpd_median
time,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-02 00:00:00+00:00,8.073125,2.485878,7.3975,215.769231,22.155645,223.186813,4.858542,0.560603,5.0425,185.312500,45.580243,181.75,10.071875,0.041977,10.070000,293.854167,79.016412,293.5,7.046250,1.435537,6.9675,140.465201,87.495178,114.959709,0.134667,0.067496,0.133417
2022-01-02 01:00:00+00:00,8.397917,2.442645,7.8050,218.481685,19.215475,223.934065,4.877083,0.535039,5.0425,184.916667,45.959599,181.75,10.065590,0.043925,10.070000,291.437500,80.655429,293.5,7.171042,1.351710,6.9775,135.301587,88.772141,92.087913,0.136063,0.067875,0.145833
2022-01-02 02:00:00+00:00,8.586667,2.274816,8.3550,221.141026,15.102800,224.769230,4.886875,0.519448,5.0425,184.520833,46.362359,181.75,10.058576,0.045686,10.070000,290.791667,81.395628,293.5,7.304375,1.188163,6.9875,130.111110,89.651223,80.109889,0.137556,0.068031,0.161833
2022-01-02 03:00:00+00:00,8.892917,2.138847,8.5875,224.148352,7.688401,225.076922,4.899792,0.496998,5.0425,184.250000,46.622375,181.75,10.051840,0.048227,10.065000,289.541667,82.124387,293.5,7.379792,1.060582,6.9875,120.206349,83.752773,77.516483,0.139340,0.069630,0.161833
2022-01-02 04:00:00+00:00,9.004375,2.052357,8.6600,225.684982,3.586566,225.164835,4.906875,0.486289,5.0425,183.916667,46.868981,181.75,10.044896,0.046522,10.056667,288.145833,82.895702,293.5,7.290208,1.080664,6.9775,108.548840,70.917851,74.468866,0.138403,0.068623,0.161833


In [14]:
univariate_features_df.shape

(4790, 27)

### 二元特征提取

In [16]:
BIVARIATE_SUMMARY_STATS = {
    "covariance": covariance,
    "co_integration": co_integration,
    "js_div": jensenshannon,
}
BIVARIATE_TRANSFORMATIONS = {
    "corr": signal.correlate,
    "conv": signal.convolve,
    "rel_entr": rel_entr,
}

# pairs of variables
colnames = buoy.columns.tolist()
col_combs = list(itertools.combinations(colnames, 2))

bivariate_features = []
# ---------------------------------------
# for each row
# ---------------------------------------
for i, _ in X.iterrows():
    # ---------------------------------------
    # feature set in the i-th time-step
    # ---------------------------------------
    feature_set_i = {}
    for col1, col2 in col_combs:
        # ---------------------------------------
        # features for pair of columns col1, col2
        # ---------------------------------------
        # getting the i-th instance for each column
        x1 = X.loc[i, X.columns.str.startswith(col1)]
        x2 = X.loc[i, X.columns.str.startswith(col2)]
        # compute each summary stat
        for feat, func in BIVARIATE_SUMMARY_STATS.items():
            feature_set_i[f"{col1}|{col2}_{feat}"] = func(x1, x2)
        # for each transformation
        for trans_feat, trans_func in BIVARIATE_TRANSFORMATIONS.items():
            # apply transformation
            x_trans = trans_func(x1, x2)
            # compute summary stat
            for feat, summary_func in SUMMARY_STATS.items():
                feature_set_i[f"{col1}|{col2}_{trans_func}_{feat}"] = summary_func(x_trans)
    bivariate_features.append(feature_set_i)
bivariate_features_df = pd.DataFrame(bivariate_features, index = X.index)

In [17]:
X_with_features = pd.concat([
    X, 
    univariate_features_df, 
    bivariate_features_df
], axis = 1)

### 数据分割

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_with_features,
    Y,
    test_size = 0.3,
    shuffle = False,
)

### 模型构建

In [19]:
model_w_fe = MultiOutputRegressor(LGBMRegressor())
model_w_fe.fit(X_train, Y_train)

MultiOutputRegressor(estimator=LGBMRegressor())

In [20]:
preds_w_fe = model_w_fe.predict(X_test)
mape(Y_test, preds_w_fe)

0.23209346352885196

## Feature Selection

In [22]:
# importance of features
avg_imp = pd.DataFrame([
    x.feature_importances_ for x in model_w_fe.estimators_
]).mean()

# top 100 features
n_top_features = 100
importance_scores = pd.Series(dict(zip(X_train.columns, avg_imp)))
top_features = importance_scores.sort_values(ascending = False)[:n_top_features]
top_features_nm = top_features.index

# 特征选择
X_train_top = X_train[top_features_nm]
X_test_top = X_test[top_features_nm]

# 模型拟合
model_top_features = MultiOutputRegressor(LGBMRegressor())
model_top_features.fit(X_train_top, Y_train)

# 模型预测
preds_top_features = model_top_features.predict(X_test_top)

# 模型评估
mape(Y_test, preds_top_features)

0.23549452747597632